In [31]:
import nltk,math
from nltk import *

In [32]:
# Reduced cess-esp data

import nltk
from nltk import *
from nltk.tag import hmm


s_tagged = nltk.corpus.cess_esp.tagged_sents()

list1 = []
list2 = []

for sent in s_tagged:
    for word, tag in sent:
        n_tag = re.findall(r'^v[a-z]{,2}|^F[a-z]{,2}|^d[a-z]{,1}|^n[a-z]{,1}|^a[a-z]{,1}|^s[a-z]{,1}|^Z[a-z]{,1}|^W[a-z]{,1}|^p[a-z]{,1}', tag)
        listToStr = ' '.join(map(str, n_tag)) 
        list2.append((word,listToStr))
    list1.append(list2)
    list2 = []

In [33]:
import math
tagged_sentences = [sentence for sentence in list1]

# hold out 20% for testing, get index for 20% split
split_idx = math.floor(len(tagged_sentences)*0.2)
testing_sentences = tagged_sentences[0:split_idx]
training_sentences = tagged_sentences[split_idx:]
TEST_EXAMPLE_SENTENCE_INDEX = 676

In [34]:
tnt_tagger = nltk.tag.tnt.TnT()
tnt_tagger.train(training_sentences)
tnt_tagger_preds = tnt_tagger.tag_sents([[word for word,_ in test_sent] for test_sent in testing_sentences])
print(tnt_tagger_preds[TEST_EXAMPLE_SENTENCE_INDEX])

[('Alberto_Fernández', 'Unk'), (',', 'Fc'), ('que', 'pr'), ('ha', 'vai'), ('presidido', 'aq'), ('la', 'da'), ('primera', 'ao'), ('reunión', 'nc'), ('del', 'sp'), ('grupo', 'nc'), ('del', 'sp'), ('PP', 'np'), ('en', 'sp'), ('el', 'da'), ('Parlament', 'Unk'), ('tras', 'sp'), ('las', 'da'), ('elecciones', 'nc'), (',', 'Fc'), ('ha', 'vai'), ('relativizado', 'Unk'), ('el', 'da'), ('hecho', 'nc'), ('de', 'sp'), ('que', ''), ('CiU', 'np'), ('haya', 'vas'), ('expresado', 'vmp'), ('sus', 'dp'), ('preferencias', 'Unk'), ('por', 'sp'), ('llegar', 'vmn'), ('a', 'sp'), ('acuerdos', 'nc'), ('con', 'sp'), ('ERC', 'np'), ('en', 'sp'), ('Cataluña', 'np'), ('y', ''), ('*0*', 'sn'), ('ha', 'vai'), ('recordado', 'vmp'), ('que', ''), ('la', 'da'), ('coalición', 'nc'), ('también', ''), ('optó', 'vmi'), ('en', 'sp'), ('primer', 'ao'), ('lugar', 'nc'), ('por', 'sp'), ('pactar', 'Unk'), ('con', 'sp'), ('Esquerra', 'Unk'), ('después', ''), ('de', 'sp'), ('los', 'da'), ('comicios', 'nc'), ('autonómicos', 'Unk'),

In [35]:
perceptron_tagger = nltk.tag.perceptron.PerceptronTagger(load=False)
perceptron_tagger.train(training_sentences)
perceptron_tagger_preds = [perceptron_tagger.tag([word for word,_ in test_sentence]) for test_sentence in testing_sentences]
print(perceptron_tagger_preds[TEST_EXAMPLE_SENTENCE_INDEX])

[('Alberto_Fernández', 'np'), (',', 'Fc'), ('que', 'pr'), ('ha', 'vai'), ('presidido', 'vmp'), ('la', 'da'), ('primera', 'ao'), ('reunión', 'nc'), ('del', 'sp'), ('grupo', 'nc'), ('del', 'sp'), ('PP', 'np'), ('en', 'sp'), ('el', 'da'), ('Parlament', 'np'), ('tras', 'sp'), ('las', 'da'), ('elecciones', 'nc'), (',', 'Fc'), ('ha', 'vai'), ('relativizado', 'vmp'), ('el', 'da'), ('hecho', 'nc'), ('de', 'sp'), ('que', ''), ('CiU', 'np'), ('haya', 'vas'), ('expresado', 'vmp'), ('sus', 'dp'), ('preferencias', 'nc'), ('por', 'sp'), ('llegar', 'vmn'), ('a', 'sp'), ('acuerdos', 'nc'), ('con', 'sp'), ('ERC', 'np'), ('en', 'sp'), ('Cataluña', 'np'), ('y', ''), ('*0*', 'sn'), ('ha', 'vai'), ('recordado', 'vmp'), ('que', ''), ('la', 'da'), ('coalición', 'nc'), ('también', ''), ('optó', 'vmi'), ('en', 'sp'), ('primer', 'ao'), ('lugar', 'nc'), ('por', 'sp'), ('pactar', 'vmn'), ('con', 'sp'), ('Esquerra', 'np'), ('después', ''), ('de', 'sp'), ('los', 'da'), ('comicios', 'nc'), ('autonómicos', 'aq'), ('y

In [36]:
from sklearn import metrics

golds = [tag for sentence in testing_sentences for _,tag in sentence]
tnt_tagger_pred_labels = [tag for sentence in tnt_tagger_preds for _,tag in sentence]
perceptron_tagger_pred_labels = [tag for sentence in perceptron_tagger_preds for _,tag in sentence]

for preds,tagger in [(tnt_tagger_pred_labels,tnt_tagger),(perceptron_tagger_pred_labels,perceptron_tagger)]:
    print("Metrics for",tagger)
    print(" Accuracy :", metrics.accuracy_score(golds,preds))
    print(" Precision:", metrics.precision_score(golds,preds,average='weighted'))
    print(" Recall   :", metrics.recall_score(golds,preds,average='weighted'))
    print(" F1-Score :", metrics.f1_score(golds,preds,average='weighted'))
    print()

Metrics for <nltk.tag.tnt.TnT object at 0x12a556910>
 Accuracy : 0.88492413549753
 Precision: 0.9834545909492033
 Recall   : 0.88492413549753
 F1-Score : 0.9230045897013717

Metrics for <nltk.tag.perceptron.PerceptronTagger object at 0x12a5566a0>
 Accuracy : 0.9735797459421313
 Precision: 0.9733551859619565
 Recall   : 0.9735797459421313
 F1-Score : 0.972926017548243



In [37]:
from nltk.tag import brill, brill_trainer 

def train_brill_tagger(initial_tagger, train_sents, **kwargs): 
    templates = [ 
            brill.Template(brill.Pos([-1])), 
            brill.Template(brill.Pos([1])), 
            brill.Template(brill.Pos([-2])), 
            brill.Template(brill.Pos([2])), 
            brill.Template(brill.Pos([-2, -1])), 
            brill.Template(brill.Pos([1, 2])), 
            brill.Template(brill.Pos([-3, -2, -1])), 
            brill.Template(brill.Pos([1, 2, 3])), 
            brill.Template(brill.Pos([-1]), brill.Pos([1])), 
            brill.Template(brill.Word([-1])), 
            brill.Template(brill.Word([1])), 
            brill.Template(brill.Word([-2])), 
            brill.Template(brill.Word([2])), 
            brill.Template(brill.Word([-2, -1])), 
            brill.Template(brill.Word([1, 2])), 
            brill.Template(brill.Word([-3, -2, -1])), 
            brill.Template(brill.Word([1, 2, 3])), 
            brill.Template(brill.Word([-1]), brill.Word([1])), 
            ] 
      
    # Using BrillTaggerTrainer to train  
    trainer = brill_trainer.BrillTaggerTrainer( 
            initial_tagger, templates, deterministic = True) 
      
    return trainer.train(train_sents, **kwargs) 

In [38]:
from nltk.tag import brill, brill_trainer 
from nltk.tag import DefaultTagger 
import itertools
from nltk.tbl import Template
from nltk.tag import brill, brill_trainer
from nltk.probability import FreqDist, ConditionalFreqDist

def backoff_tagger(train_sents, tagger_classes, backoff=None):
	for cls in tagger_classes:
		backoff = cls(train_sents, backoff=backoff)
	
	return backoff

def word_tag_model(words, tagged_words, limit=200):
	fd = FreqDist(words)
	cfd = ConditionalFreqDist(tagged_words)
	most_freq = (word for word, count in fd.most_common(limit))
	return dict((word, cfd[word].max()) for word in most_freq)

patterns = [
	(r'^\d+$', 'CD'),
	(r'.*ing$', 'VBG'), # gerunds, i.e. wondering
	(r'.*ment$', 'NN'), # i.e. wonderment
	(r'.*ful$', 'JJ') # i.e. wonderful
]

def unigram_feature_detector(tokens, index, history):
	return {'word': tokens[index]}
  
# Initializing 
default_tagger = DefaultTagger('NN') 
  
initial_tag = backoff_tagger( 
        training_sentences, [UnigramTagger, BigramTagger,  
                    TrigramTagger], backoff = default_tagger) 
      
a = initial_tag.evaluate(testing_sentences) 
print ("Accuracy of Back-off Initial Tag : ", a) 

Accuracy of Back-off Initial Tag :  0.8809324276640791


In [39]:
brill_tag = train_brill_tagger(initial_tag, training_sentences) 
b = brill_tag.evaluate(testing_sentences) 
  
print ("Accuracy of brill_tag : ", b) 

Accuracy of brill_tag :  0.883424488355681


In [40]:
initial_tag2 = nltk.HiddenMarkovModelTagger.train(training_sentences)

a = initial_tag2.evaluate(testing_sentences) 
print ("Accuracy of Hmm Initial Tag : ", a) 

Accuracy of Hmm Initial Tag :  0.9204745942131263


In [41]:
brill_tag = train_brill_tagger(initial_tag2, training_sentences) 
b = brill_tag.evaluate(testing_sentences) 
  
print ("Accuracy of brill_tag : ", b) 

Accuracy of brill_tag :  0.9225696894848271
